# Summarize Webiste Model

### I created a LLM model using ollama that summarizes any website and gives you some details about it. 

### Importing
- `os`: Access environment variables and interact with the operating system.
- `requests`: Send HTTP requests (GET, POST, etc.).
- `from dotenv import load_dotenv`: Loads environment variables from a .env file into your system environment.
- `BeautifulSoup`: Parse and extract data from HTML and XML documents.
- `from IPython.display import Markdown, display`: Rich display in Jupyter Notebooks.

In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [13]:
## testing
message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hi there! It's great to meet you for the first time! I'm Llama, your friendly AI assistant, and I'm here to help with any questions or topics you'd like to discuss. How's your day going so far? Is there something specific on your mind that you'd like to talk about, or do you just want to get to know me a bit better? I'm all ears (or rather, all text)!


## Types of prompts

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [10]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [11]:
display_summary("https://www.youtube.com/")

### Short Summary of YouTube Website

#### Purpose and Functionality

YouTube is a video-sharing platform where users can upload, share, and view videos.

#### Overview of Available Sections

The website provides information on:

* **About**: Foundational information about the company and its mission.
* **Press**: Updates and news for media professionals covering YouTube and Google.
* **Copyright**: Information on copyright policies and procedures.
* **Contact us**: Details for getting in touch with YouTube team or customer support.
* **Creator**: Resources specifically for content creators, including tutorials and guidelines.
* **Advertise**: Commercial opportunities for brands to promote their products on the platform.

#### Technical and Business Details

The website also includes information on:

* **Developers**: Documentation and APIs for developers building applications connected with YouTube.
* **Terms**: Legal agreements users must agree to when using the service.
* **Privacy**: Information on how YouTube collects, uses, and protects user data.


In [12]:
display_summary("https://www.linkedin.com/")

**Summary**

LinkedIn is a professional networking platform where users can create profiles, connect with colleagues and friends, share articles, find job opportunities, take courses, play games, and keep up-to-date on industry news.

### News Announcements

* LinkedIn announced the launch of its new "Open to Work" feature, allowing users to privately indicate that they are open to new job opportunities.
* The platform introduced a new AI-powered feature to unlock community knowledge in collaborative articles.
* Users can now discover and explore software tools for businesses on LinkedIn.

### Features and Areas

* **Job Search**: Post job openings or search for a new role with millions of people's resumes at your fingertips.
* **Learning**: Take online courses in various topics, from business development to web design, to upskill and reskill.
* **Networking**: Connect with colleagues, classmates, and friends on LinkedIn, and stay up-to-date on industry news through articles, messages, and live videos.
* **Business Solutions**: Access resources, tools, and insights for businesses of all sizes.

### Languages

LinkedIn is available in multiple languages, including Arabic, Bengali, Czech, Danish, English, Spanish, French, and more.